<a href="https://colab.research.google.com/github/Lukebotbol/MSc_Thesis_Abstractive_Summarization/blob/main/Extractive_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sumy
!pip install rouge

     |████████████████████████████████| 83 kB 1.7 MB/s 
     |████████████████████████████████| 10.1 MB 29.8 MB/s 
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21711 sha256=9a53228573d04d7e46ab09934c3eebc0c465ebdffd745701a34d26b3fd83c3e2
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746883 sha256=5330f71a54e3cce0acad916389ea0cb6d645a5a16c86e31fceb4b4f6b22ab587
  Stored in directory: /root/.cache/pip/wheels/57/e8/3f/120ccc1ff7541c108bc5d656e2a14c39da0d824653b62284c6
Successfully built breadability pycountry


In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from rouge import Rouge
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset(1).csv')

# TextRank & KL Sum

In [ ]:
documents = df['original_text']

In [ ]:
def get_summaries(SumAlgo, documents):
  sum = []
  for i in range(len(documents)):
    parser = PlaintextParser.from_string(documents[i], Tokenizer("english"))
    summarizer = SumAlgo
    summarizer.stop_words = get_stop_words("english")
    summary = summarizer(parser.document, 1)
    sents = " ".join([str(sentence) for sentence in summary])
    sum.append(sents)
  return sum

In [ ]:
#Generate summaries from different algo
stemmer = Stemmer("english")

summaries = {}
summaries['original_text'] = df['original_text']
summaries['reference_summary'] = df['reference_summary']
summaries['TextRank'] = get_summaries(TextRankSummarizer(stemmer), documents)
#summaries['LexRank'] = get_summaries(LexRankSummarizer(stemmer), documents)
summaries['KLSum'] = get_summaries(KLSummarizer(stemmer), documents)
summaries['Luhn_Sum'] = get_summaries(LuhnSummarizer(stemmer), documents)
summaries['SumBasic'] = get_summaries(SumBasicSummarizer(stemmer), documents)
summaries['LSA_Sum'] = get_summaries(LsaSummarizer(stemmer), documents)

In [ ]:
Summaries_df = pd.DataFrame(summaries)

In [ ]:
Summaries_df[151:154].to_csv('/content/drive/MyDrive/Colab Notebooks/summaries_unsup.csv')

In [ ]:
Summaries_df[151:154]

,original_text,reference_summary,TextRank,KLSum,Luhn_Sum,SumBasic,LSA_Sum
151,you also agree to register one account only fo...,service does not allow alternative accounts.,the site administrators reserve the right to b...,the site administrators reserve the right to b...,you also agree to register one account only fo...,you also agree to register one account only fo...,the information we obtain through your use of ...
152,offer you targeted advertising in order to per...,the service uses your personal data for advert...,offer you targeted advertising in order to per...,offer you targeted advertising in order to per...,offer you targeted advertising in order to per...,offer you targeted advertising in order to per...,offer you targeted advertising in order to per...
153,researchgate strives to connect the world of s...,pseudonym not allowed user to user trust is re...,it is essential for the credibility of the wor...,you may not accept adopt or post content that ...,it is essential for the credibility of the wor...,this again is essential for the credibility of...,you may not accept adopt or post content that ...


In [ ]:
rouge = Rouge()
scores_TR = rouge.get_scores(Summaries_df.TextRank, Summaries_df.reference_summary, avg=True)
scores_KL = rouge.get_scores(Summaries_df.KLSum, Summaries_df.reference_summary, avg=True)


In [ ]:
scores_TR['rouge-1']['f']

0.24646116166986476

In [ ]:
scores = {}
for i in Summaries_df.columns:
  if i != 'original_text' and i!= 'reference_summary':
    score = rouge.get_scores(Summaries_df[i], Summaries_df.reference_summary, avg=True)
    f1= []
    for z in score.keys():
      f1.append(score[z]['f'] * 100)
    scores[i] = f1

In [ ]:
scores_df = pd.DataFrame(data = scores.values(), index=scores.keys(), columns=['Rouge-1', 'Rouge-2', 'Rouge-l'])

In [ ]:
scores_df

,Rouge-1,Rouge-2,Rouge-l
TextRank,24.646116,7.805802,20.615177
KLSum,23.547594,6.810273,19.799045
Luhn_Sum,23.979510,7.488704,20.243000
SumBasic,23.666323,7.122913,20.109419
LSA_Sum,23.765487,7.501224,20.130580
